In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [3]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [4]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [5]:
outputs

[tensor([[[[1.1011, 0.1701, 0.6535,  ..., 0.9833, 0.0562, 0.4889],
           [0.4669, 0.0000, 0.5474,  ..., 0.7028, 1.0510, 1.0566],
           [0.3859, 0.8565, 0.1349,  ..., 1.1262, 0.0361, 0.5914],
           ...,
           [0.5407, 0.5156, 0.2273,  ..., 0.5114, 1.3676, 0.8280],
           [1.8092, 0.2452, 0.5215,  ..., 0.7771, 0.5566, 0.7237],
           [0.2761, 0.0000, 0.6211,  ..., 0.7444, 1.0347, 0.6604]],
 
          [[0.2299, 0.4488, 0.0000,  ..., 0.2837, 0.8636, 0.5749],
           [0.2186, 0.5569, 0.5434,  ..., 0.0000, 1.2134, 0.8057],
           [0.9141, 1.2716, 0.9865,  ..., 0.2843, 0.5603, 0.7815],
           ...,
           [0.1236, 0.9184, 0.0000,  ..., 0.6594, 0.0000, 0.5045],
           [0.0753, 0.0894, 0.0000,  ..., 0.2836, 0.6286, 0.4231],
           [0.1276, 0.9504, 0.0000,  ..., 0.8532, 0.2357, 0.7564]],
 
          [[0.3295, 0.7396, 0.8178,  ..., 1.5668, 1.0576, 0.8193],
           [0.4445, 0.7561, 0.8111,  ..., 0.0041, 0.5870, 1.2543],
           [0.4527, 0.43

In [6]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [7]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [8]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


